In [1]:
# Importando bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder

In [2]:
# Lendo o csv
df = pd.read_csv('dataset_teste_selecao.csv')
df.head()

,paciente_id,coriza,tosse_seca_ou_produtiva,calafrios,febre,dispneia,fadiga,anorexia,mialgia,astenia,...,idade,cor_autorreferida,estado_civil,escolaridade,tipo_caso_à_admissão,tempo_sintomas_internacao,desfecho,obito_menos24horas,obito_vm,obito_uti
0,e5841df2166dd424a57127423d276bbe,Não,Não,Não,Não,Não,Não,Não,Não,Não,...,76.0,Parda/mulata,Casado/ amasiado,Ensino fundamental incompleto(1ºG),Caso suspeito,NaN,obito,Não,Sim,Sim
1,3c59dc048e8850243be8079a5c74d079,Não,Sim,Não,Sim,Sim,Não,Não,Sim,Não,...,33.0,Não informado,Não informado,Não informado,Caso suspeito,7.0,alta,NaN,NaN,NaN
2,99c5e07b4d5de9d18c350cdf64c5aa3d,Não,Não,Não,Sim,Sim,Não,Não,Não,Não,...,42.0,Amarela/Oriental,Casado/ amasiado,Ensino médio completo,Caso suspeito,10.0,alta,NaN,NaN,NaN
3,1587965fb4d4b5afe8428a4a024feb0d,Não,Não,Não,Não,Sim,Não,Não,Não,Não,...,64.0,Branca,Casado/ amasiado,Ensino fundamental incompleto(1ºG),Caso suspeito,11.0,obito,Não,Sim,Sim
4,b4a528955b84f584974e92d025a75d1f,Não,Não,Não,Não,Sim,Não,Não,Não,Não,...,73.0,Parda/mulata,Casado/ amasiado,Ensino fundamental incompleto(1ºG),Caso suspeito,7.0,obito,Não,Sim,Sim


In [3]:
# Selecionando atributos categóricos
dataCategorico = df.drop(['paciente_id','tempo_vmi_total', 'tempo_uti_total','tempo_internacao','tempo_referencia_internacao','idade','tempo_sintomas_internacao'], axis=1)

In [4]:
# Aplicando o label enconder
encoder = LabelEncoder()
for i in range(0,len(dataCategorico.columns)):
    dataCategorico[dataCategorico.columns[i]] = encoder.fit_transform(dataCategorico[dataCategorico.columns[i]])

In [5]:
# Adicionando identificação do paciente aos atributos categóricos
dataCategorico['paciente_id'] = df['paciente_id']

In [6]:
# Selecionando atributos numéricos
dataNumerico= df.loc[:,['paciente_id','tempo_vmi_total', 'tempo_uti_total','tempo_internacao','tempo_referencia_internacao','idade','tempo_sintomas_internacao']]

In [7]:
# Aplicando merge entre os datasets
base2 = dataCategorico.merge(dataNumerico, left_on='paciente_id', right_on='paciente_id')

# Atualizando dataset principal
df = base2
df = df.drop('paciente_id',axis=1)

In [8]:
# Removendo características vazias
df = df.dropna()

In [9]:
# Criando um dataframe somente com as features
X = df.drop('desfecho',axis=1)

# Criando um dataframe somente com as classes
y = df['desfecho']

# Definindo as métricas de avaliação
scoring = ['accuracy', 'precision','recall', 'f1', 'roc_auc']

# Variaveis para armazenar o melhor classificador de cada modelo
knn_best_estimator = ""
knn_best_score = 0
nb_best_estimator = ""
nb_best_score = 0
nb_distribution = ""
dt_best_estimator = ""
dt_best_score = 0
rf_best_estimator = ""
rf_best_score = 0
mlp_best_estimator = ""
mlp_best_score = 0
svc_best_estimator = ""
svc_best_score = 0
svc_kernel = ""

## KNN

In [10]:
# Instanciando o classificador
knn = KNeighborsClassifier(n_jobs=-1)

# Definindo alguns parametros para teste
params = {
    'n_neighbors': [2,3,5,10,20],
    'metric': ['euclidean','manhattan','chebyshev']
}

In [11]:
# Instanciando o gridsearch e executando o modelo

for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=knn,
                           param_grid=params,
                           cv = 4,
                           n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (knn_best_score < grid_search.best_score_):
        knn_best_estimator = best
        knn_best_score = grid_search.best_score_
    

Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 15 candidates, totalling 60 fits
Melhor classificador:  KNeighborsClassifier(metric='euclidean', n_jobs=-1)
Score:  0.8979591836734694

Métrica de avaliacao:  precision
Fitting 4 folds for each of 15 candidates, totalling 60 fits
Melhor classificador:  KNeighborsClassifier(metric='euclidean', n_jobs=-1, n_neighbors=2)
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 15 candidates, totalling 60 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Melhor classificador:  KNeighborsClassifier(metric='euclidean', n_jobs=-1, n_neighbors=2)
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 15 candidates, totalling 60 fits
Melhor classificador:  KNeighborsClassifier(metric='euclidean', n_jobs=-1, n_neighbors=2)
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 15 candidates, totalling 60 fits
Melhor classificador:  KNeighborsClassifier(metric='euclidean', n_jobs=-1, n_neighbors=2)
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


## Naive Bayes

In [12]:
nb = GaussianNB()
params = {'var_smoothing': np.logspace(0,-9, num=10)}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=nb,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (nb_best_score < grid_search.best_score_):
        nb_best_estimator = best
        nb_best_score = grid_search.best_score_
        nb_distribution = 'Gaussian'

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Melhor classificador:  GaussianNB(var_smoothing=1e-05)
Score:  0.9846938775510203

Métrica de avaliacao:  precision
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Melhor classificador:  GaussianNB(var_smoothing=1.0)
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Melhor classificador:  GaussianNB(var_smoothing=1.0)
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Melhor classificador:  GaussianNB(var_smoothing=1.0)
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 10 candidates, totalling 40 fits
Melhor classificador:  GaussianNB(var_smoothing=1.0)
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [13]:
nb = MultinomialNB()
params = {
    'alpha': [0.1,0.2,0.3,0.5,0.7,1.0]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=nb,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (nb_best_score < grid_search.best_score_):
        nb_best_estimator = best
        nb_best_score = grid_search.best_score_
        nb_distribution = 'Multinomial'

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  MultinomialNB()
Score:  0.6887755102040817

Métrica de avaliacao:  precision
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  MultinomialNB(alpha=0.1)
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  MultinomialNB(alpha=0.1)
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  MultinomialNB(alpha=0.1)
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  MultinomialNB(alpha=0.1)
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


In [14]:
nb = BernoulliNB()
params = {
    'alpha': [0.1,0.2,0.3,0.5,0.7,1.0]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=nb,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (nb_best_score < grid_search.best_score_):
        nb_best_estimator = best
        nb_best_score = grid_search.best_score_
        nb_distribution = 'Bernoulli'

Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  0.9693877551020409

Métrica de avaliacao:  precision
Fitting 4 folds for each of 6 candidates, totalling 24 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 6 candidates, totalling 24 fits
Melhor classificador:  BernoulliNB(alpha=0.1)
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


## Decision Tree

In [15]:
dt = DecisionTreeClassifier(random_state=42)
params = {
    'criterion': ['gini','entropy'],
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=dt,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (dt_best_score < grid_search.best_score_):
        dt_best_estimator = best
        dt_best_score = grid_search.best_score_

Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 60 candidates, totalling 240 fits
Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=5, random_state=42)
Score:  1.0

Métrica de avaliacao:  precision
Fitting 4 folds for each of 60 candidates, totalling 240 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=5, random_state=42)
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 60 candidates, totalling 240 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=5, random_state=42)
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 60 candidates, totalling 240 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(


Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=5, random_state=42)
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 60 candidates, totalling 240 fits
Melhor classificador:  DecisionTreeClassifier(max_depth=2, min_samples_leaf=5, random_state=42)
Score:  nan



## Random Forest

In [16]:
rf = RandomForestClassifier(random_state=42)
params = {
    'criterion': ['gini','entropy'],
    'max_depth': [2,3,5,10,20],
    'min_samples_leaf': [5,10,20,50,100,200],
    'n_estimators': [10,25,30,50,100,200]
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=rf,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (rf_best_score < grid_search.best_score_):
        rf_best_estimator = best
        rf_best_score = grid_search.best_score_

Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 360 candidates, totalling 1440 fits
Melhor classificador:  RandomForestClassifier(max_depth=5, min_samples_leaf=5, n_estimators=10,
                       random_state=42)
Score:  0.9132653061224489

Métrica de avaliacao:  precision
Fitting 4 folds for each of 360 candidates, totalling 1440 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan n

Melhor classificador:  RandomForestClassifier(max_depth=2, min_samples_leaf=5, n_estimators=10,
                       random_state=42)
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 360 candidates, totalling 1440 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan n

Melhor classificador:  RandomForestClassifier(max_depth=2, min_samples_leaf=5, n_estimators=10,
                       random_state=42)
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 360 candidates, totalling 1440 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan n

Melhor classificador:  RandomForestClassifier(max_depth=2, min_samples_leaf=5, n_estimators=10,
                       random_state=42)
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 360 candidates, totalling 1440 fits
Melhor classificador:  RandomForestClassifier(max_depth=2, min_samples_leaf=5, n_estimators=10,
                       random_state=42)
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan n

## MLP

In [17]:
mlp = MLPClassifier(max_iter=100)
params = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=mlp,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (mlp_best_score < grid_search.best_score_):
        mlp_best_estimator = best
        mlp_best_score = grid_search.best_score_

Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 32 candidates, totalling 128 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhor classificador:  MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 30, 10),
              learning_rate='adaptive', max_iter=100)
Score:  0.8979591836734694

Métrica de avaliacao:  precision
Fitting 4 folds for each of 32 candidates, totalling 128 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhor classificador:  MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 30, 10), max_iter=100,
              solver='sgd')
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 32 candidates, totalling 128 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhor classificador:  MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 30, 10), max_iter=100,
              solver='sgd')
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 32 candidates, totalling 128 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhor classificador:  MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 30, 10), max_iter=100,
              solver='sgd')
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 32 candidates, totalling 128 fits
Melhor classificador:  MLPClassifier(activation='tanh', hidden_layer_sizes=(10, 30, 10), max_iter=100,
              solver='sgd')
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


## SVM

In [18]:
#kernel rbf
svc_kernel = 'rbf'
svc = SVC(kernel='rbf')
params = {'C': [0.1, 1, 10, 100], 
          'gamma': [1,0.1, 0.01, 0.001]}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=svc,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (svc_best_score < grid_search.best_score_):
        svc_best_estimator = best
        svc_best_score = grid_search.best_score_
        

Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 16 candidates, totalling 64 fits
Melhor classificador:  SVC(C=0.1, gamma=1)
Score:  0.8928571428571429

Métrica de avaliacao:  precision
Fitting 4 folds for each of 16 candidates, totalling 64 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Melhor classificador:  SVC(C=0.1, gamma=1)
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 16 candidates, totalling 64 fits
Melhor classificador:  SVC(C=0.1, gamma=1)
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 16 candidates, totalling 64 fits
Melhor classificador:  SVC(C=0.1, gamma=1)
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 16 candidates, totalling 64 fits
Melhor classificador:  SVC(C=0.1, gamma=1)
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [19]:
#kernel linear
svc_kernel = 'linear'
svc = SVC(kernel='linear')
params = {'C': [0.1,1,10]}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=svc,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (svc_best_score < grid_search.best_score_):
        svc_best_estimator = best
        svc_best_score = grid_search.best_score_

Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 3 candidates, totalling 12 fits
Melhor classificador:  SVC(C=1, kernel='linear')
Score:  0.6887755102040816

Métrica de avaliacao:  precision
Fitting 4 folds for each of 3 candidates, totalling 12 fits
Melhor classificador:  SVC(C=0.1, kernel='linear')
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 3 candidates, totalling 12 fits
Melhor classificador:  SVC(C=0.1, kernel='linear')
Score:  nan

Métrica de avaliacao:  f1
Fitting 4 folds for each of 3 candidates, totalling 12 fits
Melhor classificador:  SVC(C=0.1, kernel='linear')
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 3 candidates, totalling 12 fits
Melhor classificador:  SVC(C=0.1, kernel='linear')
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan]
  warnings.warn(


In [20]:
#kernel polinomial
svc_kernel = 'polinomial'
svc = SVC(kernel='poly')
params = {'C': [0.1, 1, 10], 
          'gamma': [1,0.1,0.01],
          'degree': [3,4]}
for score_item in scoring:
    print('Métrica de avaliacao: ', score_item)
    grid_search = GridSearchCV(estimator=svc,
                            param_grid=params,
                            cv = 4,
                            n_jobs=-1, verbose=1, scoring=score_item)
    #%%time
    grid_search.fit(X, y)
    best = grid_search.best_estimator_
    print('Melhor classificador: ',best)
    print('Score: ',grid_search.best_score_)
    print()
    if (svc_best_score < grid_search.best_score_):
        svc_best_estimator = best
        svc_best_score = grid_search.best_score_
        

Métrica de avaliacao:  accuracy
Fitting 4 folds for each of 18 candidates, totalling 72 fits
Melhor classificador:  SVC(C=0.1, degree=4, gamma=1, kernel='poly')
Score:  0.7857142857142858

Métrica de avaliacao:  precision
Fitting 4 folds for each of 18 candidates, totalling 72 fits
Melhor classificador:  SVC(C=0.1, gamma=1, kernel='poly')
Score:  nan

Métrica de avaliacao:  recall
Fitting 4 folds for each of 18 candidates, totalling 72 fits
Melhor classificador:  SVC(C=0.1, gamma=1, kernel='poly')
Score:  nan

Métrica de avaliacao:  f1


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


Fitting 4 folds for each of 18 candidates, totalling 72 fits
Melhor classificador:  SVC(C=0.1, gamma=1, kernel='poly')
Score:  nan

Métrica de avaliacao:  roc_auc
Fitting 4 folds for each of 18 candidates, totalling 72 fits
Melhor classificador:  SVC(C=0.1, gamma=1, kernel='poly')
Score:  nan



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
